In [1]:
!rm -rf yolov5  # Remove the incomplete directory
!git clone https://github.com/ultralytics/yolov5
# !cd yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 17120, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 17120 (delta 47), reused 22 (delta 18), pack-reused 17043 (from 2)
Receiving objects: 100% (17120/17120), 15.78 MiB | 31.44 MiB/s, done.
Resolving deltas: 100% (11746/11746), done.


In [2]:
%cd yolov5

/kaggle/working/yolov5


In [20]:
# import shutil
# import os

# # Path to the Kaggle working directory
# output_path = "/kaggle/working/"

# # Clear all files and folders in the output directory
# for filename in os.listdir(output_path):
#     file_path = os.path.join(output_path, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)  # Remove file or symlink
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)  # Remove directory
#     except Exception as e:
#         print(f"Failed to delete {file_path}. Reason: {e}")

# print("Output directory cleared.")


Output directory cleared.


In [3]:
!pip install -r /kaggle/working/yolov5/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 16.7 MB/s eta 0:00:00a 0:00:01


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
dataset_path = "/kaggle/input/cholecseg8k"
output_path = "/kaggle/working/"

# Define class mapping (adjust as per dataset)
class_mapping = {
    "background": 0,
    "abdominal_wall": 1,
    "liver": 2,
    "gastrointestinal_tract": 3,
    "fat": 4,
    "grasper": 5,
    "connective_tissue": 6,
    "blood": 7,
    "cystic_duct": 8,
    "L_hook_electrocautery": 9,
    "gallbladder": 10,
    "hepatic_vein": 11,
    "liver_ligament": 12,
}

# Ensure output directories exist
os.makedirs(os.path.join(output_path, "images/train"), exist_ok=True)
os.makedirs(os.path.join(output_path, "images/val"), exist_ok=True)
os.makedirs(os.path.join(output_path, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(output_path, "labels/val"), exist_ok=True)

# Helper function to convert bounding boxes to YOLO format
def convert_bbox(image_size, bbox):
    dw = 1.0 / image_size[1]
    dh = 1.0 / image_size[0]
    x = (bbox[0] + bbox[2]) / 2.0
    y = (bbox[1] + bbox[3]) / 2.0
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    return x * dw, y * dh, w * dw, h * dh

# Function to parse annotation masks and extract bounding boxes
def parse_annotations(annotation_path, class_mapping):
    mask = cv2.imread(annotation_path, cv2.IMREAD_GRAYSCALE)
    bboxes = []
    for class_name, class_id in class_mapping.items():
        # Extract binary mask for the class
        class_mask = (mask == class_id).astype(np.uint8)
        # Find contours (bounding boxes) for the class
        contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 0 and h > 0:  # Avoid zero-area boxes
                bboxes.append({"class_id": class_id, "bbox": [x, y, x + w, y + h]})
    return bboxes

# Collect all image paths and corresponding annotation paths
image_paths = []
annotation_paths = []
for video_dir in os.listdir(dataset_path):
    video_path = os.path.join(dataset_path, video_dir)
    for frame_dir in os.listdir(video_path):
        frame_path = os.path.join(video_path, frame_dir)
        for file in os.listdir(frame_path):
            if file.endswith("_endo.png"):  # Raw image
                img_path = os.path.join(frame_path, file)
                annotation_file = file.replace("_endo.png", "_endo_mask.png")
                annotation_path = os.path.join(frame_path, annotation_file)
                
                if os.path.exists(annotation_path):
                    image_paths.append(img_path)
                    annotation_paths.append(annotation_path)

# Perform train-test split (80% train, 20% val)
train_images, val_images, train_annotations, val_annotations = train_test_split(
    image_paths, annotation_paths, test_size=0.2, random_state=42
)

# Process images and annotations for train and val sets
def process_dataset(image_paths, annotation_paths, split):
    for img_path, annotation_path in zip(image_paths, annotation_paths):
        # Load image and annotation
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue

        height, width, _ = img.shape
        bboxes = parse_annotations(annotation_path, class_mapping)

        # Skip if no bounding boxes found
        if not bboxes:
            print(f"No bounding boxes found for: {annotation_path}")
            continue

        # Write labels to YOLO format
        label_file = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
        label_path = os.path.join(output_path, f"labels/{split}", label_file)
        with open(label_path, "w") as f:
            for bbox in bboxes:
                class_id = bbox["class_id"]
                x, y, w, h = convert_bbox((height, width), bbox["bbox"])
                f.write(f"{class_id} {x} {y} {w} {h}\n")

        # Copy image to output folder
        output_img_path = os.path.join(output_path, f"images/{split}", os.path.basename(img_path))
        cv2.imwrite(output_img_path, img)

# Process train and val datasets
process_dataset(train_images, train_annotations, "train")
process_dataset(val_images, val_annotations, "val")

print("Dataset preparation completed!")


Dataset preparation completed!


In [5]:
# Save the modified cholecseg8k.yaml file in the Kaggle working directory
dataset_yaml = """
train: /kaggle/working/images/train
val: /kaggle/working/images/val

nc: 13  # Number of classes
names: ['background', 'abdominal_wall', 'liver', 'gastrointestinal_tract', 'fat', 'grasper', 'connective_tissue',
        'blood', 'cystic_dut', 'L_hook_electrocautery', 'gallbladder', 'hepatic_vein', 'liver_ligament']
"""

# Write the YAML content to a file
with open('/kaggle/working/cholecseg8k.yaml', 'w') as yaml_file:
    yaml_file.write(dataset_yaml)

print("YAML file saved successfully.")


YAML file saved successfully.


In [6]:
import os

# Disable W&B visualization by setting the environment variable
os.environ["WANDB_MODE"] = "disabled"


In [10]:
!python train.py --img 640 --batch 16 --epochs 25 --data /kaggle/working/cholecseg8k.yaml --weights yolov5s.pt


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-01-12 12:29:09.482622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 12:29:09.504357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 12:29:09.511560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/kaggle/working/cholecseg8k.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noauto